In [270]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale,Normalizer,OneHotEncoder
from sklearn.preprocessing import normalize
from keras.models import Sequential,load_model
from tensorflow.python.keras.saving.saved_model import load as saved_model_load
from keras.layers import Dense, Dropout, Flatten,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
import keras.optimizers 

In [271]:
train = pd.read_csv('task3.training.csv')

In [272]:
test = pd.read_csv('task3.test.WITHOUT.answers.csv')

In [273]:
train_answer = train['class']

In [274]:
train['class']

0       B
1       J
2       H
3       G
4       G
       ..
9995    F
9996    B
9997    E
9998    H
9999    I
Name: class, Length: 10000, dtype: object

In [275]:
train = train.drop(columns = 'class')

In [276]:
train = np.array(train,dtype=np.float32)
test = np.array(test,dtype=np.float32)

In [277]:
meanlist=[]
stdlist =[]

In [278]:
# for i in range(np.shape(train)[1]):
#     tmp = np.sum(train[:,i])
#     mean = tmp/np.shape(train)[0]
#     meanlist.append(mean)
#     std = np.std(train[:,i])
#     stdlist.append(std)
#     for j in range(np.shape(train)[0]):
#         train[j][i] = (train[j][i]-mean)/std

In [279]:
meanlist

[]

In [280]:
stdlist

[]

In [281]:
# normalizer = Normalizer().fit(train) 
# train = normalizer.transform(train)

In [282]:
np.shape(train[:,1])

(10000,)

In [283]:
ans = pd.get_dummies(train_answer)

In [284]:
ans = np.array(ans)

In [285]:
train_feature_len = np.shape(train)[1]

In [286]:
train_x,val_x,train_y,val_y = train_test_split(train,ans,test_size=0.3,random_state=44)

In [287]:
np.shape(train_x)

(7000, 23)

In [288]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [289]:
lr= 1e-2
opti =keras.optimizers.Adam(learning_rate=lr)
model3 = Sequential()  
model3.add(Dense(train_feature_len*1, activation = 'relu',use_bias=True))
#model3.add(BatchNormalization())
model3.add(Dense(train_feature_len*2, activation = 'relu',use_bias=True))
#model3.add(BatchNormalization())
model3.add(Dense(train_feature_len*3, activation = 'relu',use_bias=True))
#model3.add(BatchNormalization())
model3.add(Dense(train_feature_len*3, activation = 'relu',use_bias=True))
#model3.add(BatchNormalization())
model3.add(Dense(train_feature_len*1, activation = 'relu',use_bias=True))
#model3.add(BatchNormalization())
model3.add(Dense(10, activation = 'softmax',use_bias=True))
para3 = model3.compile(optimizer = opti, loss = 'categorical_crossentropy', metrics = ['acc',f1_m,precision_m, recall_m])


In [290]:
# model3=load_model('ML_task2.h5')

In [291]:
# checkpoint
filepath="weights_with_normalize.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,
mode='max')
callbacks_list = [checkpoint]

In [292]:
#model3 = load_model('weights.best.hdf5',custom_objects={'f1_m':f1_m,'precision_m':precision_m,'recall_m':recall_m})
# model3 = load_model('weights_with_normalize.best.hdf5',custom_objects={'f1_m':f1_m,'precision_m':precision_m,'recall_m':recall_m})


In [293]:
# print('Training ------------')
# model3.fit(train_x, train_y, validation_split=0.33, epochs=1500,batch_size=128,callbacks=callbacks_list,verbose=0) #2000

In [294]:
print('Training ------------')
model3.fit(train_x, train_y,  epochs=1500,batch_size=128)

Training ------------
Epoch 1/1500
55/55 [==============================] - 0s 1ms/step - loss: 1.8712 - acc: 0.2559 - f1_m: 0.1671 - precision_m: 0.5852 - recall_m: 0.1017
Epoch 2/1500
55/55 [==============================] - 0s 2ms/step - loss: 0.8808 - acc: 0.6141 - f1_m: 0.5440 - precision_m: 0.6855 - recall_m: 0.4668
Epoch 3/1500
55/55 [==============================] - 0s 2ms/step - loss: 0.5208 - acc: 0.7784 - f1_m: 0.7761 - precision_m: 0.7847 - recall_m: 0.7678
Epoch 4/1500
55/55 [==============================] - 0s 2ms/step - loss: 0.4235 - acc: 0.8189 - f1_m: 0.8187 - precision_m: 0.8215 - recall_m: 0.8160
Epoch 5/1500
55/55 [==============================] - 0s 2ms/step - loss: 0.4602 - acc: 0.7980 - f1_m: 0.7978 - precision_m: 0.7998 - recall_m: 0.7958
Epoch 6/1500
55/55 [==============================] - 0s 2ms/step - loss: 0.3749 - acc: 0.8361 - f1_m: 0.8355 - precision_m: 0.8368 - recall_m: 0.8342
Epoch 7/1500
55/55 [==============================] - 0s 2ms/step - loss

KeyboardInterrupt: 

In [243]:
# model3.save('ML_task2.h5')

In [244]:
# # model3 = load_model('weights.best.hdf5',custom_objects={'f1_m':f1_m,'precision_m':precision_m,'recall_m':recall_m})
# model3 = load_model('weights_with_normalize.best.hdf5',custom_objects={'f1_m':f1_m,'precision_m':precision_m,'recall_m':recall_m})


In [262]:
print('\nTesting ------------')
loss, accuracy, f1_score, precision, recall = model3.evaluate(val_x, val_y, verbose=0)
print('\ntest loss: ', loss)
print('\ntest f1_score: ', f1_score) #no normal best 0.9746232628822327 


Testing ------------

test loss:  0.8261266946792603

test f1_score:  0.636098325252533


In [263]:
test.shape

(2000, 23)

In [80]:
for i in range(np.shape(test)[1]):
    mean = meanlist[i]
    std  = stdlist[i]
    for j in range(np.shape(test)[0]):
        test[j][i] = (test[j][i]-mean)/std

In [81]:
np.sum(test[:,1])

20549.914

In [82]:
prediction = model3.predict(test)

In [83]:
np.argmax(prediction[0], axis=0)

0

In [84]:
alph = ['A','B','C','D','E','F','G','H','I','J']

In [85]:
prediction_final=[]

In [86]:
for i in range(np.shape(prediction)[0]):
    index = np.argmax(prediction[i], axis=0)
    prediction_final.append(alph[index])

In [87]:
prediction_final

['A',
 'G',
 'C',
 'H',
 'D',
 'H',
 'C',
 'H',
 'B',
 'D',
 'D',
 'F',
 'J',
 'E',
 'F',
 'G',
 'C',
 'C',
 'J',
 'G',
 'F',
 'E',
 'C',
 'I',
 'H',
 'B',
 'E',
 'F',
 'H',
 'G',
 'H',
 'A',
 'H',
 'J',
 'E',
 'A',
 'J',
 'G',
 'C',
 'F',
 'B',
 'J',
 'F',
 'J',
 'E',
 'D',
 'F',
 'B',
 'I',
 'C',
 'C',
 'E',
 'A',
 'A',
 'H',
 'G',
 'H',
 'D',
 'A',
 'I',
 'C',
 'B',
 'J',
 'J',
 'C',
 'I',
 'J',
 'J',
 'A',
 'G',
 'H',
 'E',
 'B',
 'H',
 'A',
 'H',
 'E',
 'E',
 'A',
 'E',
 'I',
 'G',
 'C',
 'H',
 'E',
 'H',
 'J',
 'E',
 'F',
 'D',
 'A',
 'C',
 'D',
 'F',
 'B',
 'J',
 'F',
 'F',
 'I',
 'A',
 'C',
 'F',
 'G',
 'C',
 'J',
 'J',
 'A',
 'F',
 'C',
 'I',
 'J',
 'E',
 'H',
 'J',
 'E',
 'C',
 'C',
 'B',
 'H',
 'D',
 'E',
 'E',
 'H',
 'I',
 'G',
 'A',
 'B',
 'A',
 'D',
 'H',
 'C',
 'D',
 'I',
 'I',
 'B',
 'I',
 'C',
 'G',
 'B',
 'I',
 'D',
 'F',
 'B',
 'E',
 'F',
 'A',
 'C',
 'J',
 'H',
 'B',
 'A',
 'A',
 'F',
 'J',
 'B',
 'E',
 'I',
 'B',
 'J',
 'C',
 'F',
 'D',
 'I',
 'C',
 'A',
 'F',
 'C'

In [88]:
tt = dict([('id',np.array(range(1,2001))),('Predicted',prediction_final)])

In [89]:
tt = pd.DataFrame(tt)

In [90]:
tt.to_csv('test.csv',index=False)